In [1]:
import os
import json
import torch
import numpy as np
import torchvision
import pickle as pkl
from pprint import pprint
from torchvision import ops
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from torch.utils.data import Dataset, DataLoader
from meter import DetectionAPMeter, compute_per_class_ap_as_auc

In [2]:
with open('save_data.pkl', 'rb') as fp:
    save_data = pkl.load(file=fp)

In [3]:
all_boxes = []
all_labels = []
all_scores = []
all_tp_masks = []

for detection in save_data:
    num_dets = len(detection['scores'])
    for i in range(num_dets):
        all_boxes.append(detection['boxes'][i])
        all_labels.append(detection['labels'][i])
        all_scores.append(detection['scores'][i])
        all_tp_masks.append(detection['binary_labels'][i])

all_boxes = torch.stack(all_boxes)
all_labels = torch.tensor(all_labels)
all_scores = torch.tensor(all_scores, dtype=torch.float64)
all_tp_masks = torch.tensor(all_tp_masks)

unique_lables = all_labels.unique()

In [4]:
with open('hico_20160224_det/test_hico.json') as fp:
    train_val = json.load(fp=fp)

label_map = {i.item(): j for i, j in zip(unique_lables, list(range(len(unique_lables))))}
label_map_vec = np.vectorize(label_map.get)

num_gt = torch.zeros(80, dtype=int)
for ann in train_val:
    box_anns = ann['annotations']
    for box in box_anns:
        gt_label = box['category_id']
        gt_label_mapped = label_map[gt_label]
        num_gt[gt_label_mapped] += 1

In [5]:
pprint(label_map)

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 13: 11,
 14: 12,
 15: 13,
 16: 14,
 17: 15,
 18: 16,
 19: 17,
 20: 18,
 21: 19,
 22: 20,
 23: 21,
 24: 22,
 25: 23,
 27: 24,
 28: 25,
 31: 26,
 32: 27,
 33: 28,
 34: 29,
 35: 30,
 36: 31,
 37: 32,
 38: 33,
 39: 34,
 40: 35,
 41: 36,
 42: 37,
 43: 38,
 44: 39,
 46: 40,
 47: 41,
 48: 42,
 49: 43,
 50: 44,
 51: 45,
 52: 46,
 53: 47,
 54: 48,
 55: 49,
 56: 50,
 57: 51,
 58: 52,
 59: 53,
 60: 54,
 61: 55,
 62: 56,
 63: 57,
 64: 58,
 65: 59,
 67: 60,
 70: 61,
 72: 62,
 73: 63,
 74: 64,
 75: 65,
 76: 66,
 77: 67,
 78: 68,
 79: 69,
 80: 70,
 81: 71,
 82: 72,
 84: 73,
 85: 74,
 86: 75,
 87: 76,
 88: 77,
 89: 78,
 90: 79}


In [28]:
def auc(precision, recall: torch.Tensor):
    ap = torch.tensor(0, dtype=torch.float64)
    max_rec = recall[-1]
    for i in range(recall.numel()):
        if recall[i] >= max_rec:
            break
        if recall[i] - recall[i - 1] == 0:
            continue
        if i == 0:
            ap += precision[i] * recall[i]
        else:
            ap += 0.5 * (precision[i] + precision[i - 1]) * (recall[i] - recall[i - 1])
    return ap

In [29]:
ap_list = []
precs, recalls = [], []

for i, label in enumerate(unique_lables):
    label_mask = all_labels == label
    
    boxes = all_boxes[label_mask]
    scores = all_scores[label_mask]
    tp_masks = all_tp_masks[label_mask]

    sort_idx = scores.argsort(descending=True)

    boxes_sorted = boxes[sort_idx]
    scores_sorted = scores[sort_idx]
    tp_masks_sorted = tp_masks[sort_idx]

    total_tp = tp_masks_sorted.sum()
    fp = 1 - tp_masks_sorted

    # precision = tp_masks_sorted.cumsum(0) / (tp_masks_sorted.cumsum(0) + fp.cumsum(0))
    precision = (tp_masks_sorted.cumsum(0).to(torch.float64) / torch.ones(len(tp_masks_sorted)).cumsum(0).to(torch.float64)).to(torch.float64)
    recall = (tp_masks_sorted.cumsum(0).to(torch.float64) / num_gt[i]).to(torch.float64)
    # print(precision.dtype)
    precs.append(precision)
    recalls.append(recall)

    ap = auc(precision, recall)
    ap_list.append(ap)

aps = torch.tensor(ap_list)
print(aps)
aps.mean()

tensor([0.3635, 0.2763, 0.1595, 0.2645, 0.4005, 0.4064, 0.4683, 0.3123, 0.2414,
        0.0088, 0.5267, 0.0862, 0.1576, 0.3932, 0.2055, 0.2816, 0.2840, 0.3419,
        0.3127, 0.3578, 0.4403, 0.2159, 0.4104, 0.5977, 0.1134, 0.2907, 0.0421,
        0.2199, 0.2082, 0.3536, 0.1855, 0.2196, 0.2562, 0.2508, 0.2590, 0.1134,
        0.3568, 0.3619, 0.2733, 0.0750, 0.1360, 0.0264, 0.0258, 0.0473, 0.0466,
        0.0207, 0.1480, 0.2098, 0.2023, 0.1372, 0.2047, 0.0977, 0.1373, 0.3205,
        0.2326, 0.4069, 0.0393, 0.3620, 0.0597, 0.4921, 0.1714, 0.5164, 0.1255,
        0.3487, 0.1293, 0.1063, 0.2206, 0.1164, 0.1535, 0.0546, 0.1304, 0.0360,
        0.2004, 0.1092, 0.1482, 0.1537, 0.1302, 0.4048, 0.2068, 0.3328],
       dtype=torch.float64)


tensor(0.2280, dtype=torch.float64)

In [27]:
ap_list = []
precs, recalls = [], []

for i, label in enumerate(unique_lables):
    label_mask = all_labels == label
    
    boxes = all_boxes[label_mask]
    scores = all_scores[label_mask]
    tp_masks = all_tp_masks[label_mask]

    sort_idx = scores.argsort(descending=True)

    boxes_sorted = boxes[sort_idx]
    scores_sorted = scores[sort_idx]
    tp_masks_sorted = tp_masks[sort_idx]

    total_tp = tp_masks_sorted.sum()
    fp = 1 - tp_masks_sorted

    # precision = tp_masks_sorted.cumsum(0) / (tp_masks_sorted.cumsum(0) + fp.cumsum(0))
    precision = (tp_masks_sorted.cumsum(0).to(torch.float64) / torch.ones(len(tp_masks_sorted)).cumsum(0).to(torch.float64)).to(torch.float64)
    recall = (tp_masks_sorted.cumsum(0).to(torch.float64) / num_gt[i]).to(torch.float64)
    # print(precision.dtype)
    precs.append(precision)
    recalls.append(recall)

    ap = compute_per_class_ap_as_auc((precision, recall))
    ap_list.append(ap)

aps = torch.tensor(ap_list)
print(aps)
aps.mean()

tensor([0.3635, 0.2763, 0.1595, 0.2645, 0.4005, 0.4064, 0.4683, 0.3123, 0.2414,
        0.0088, 0.5267, 0.0862, 0.1576, 0.3932, 0.2055, 0.2816, 0.2840, 0.3419,
        0.3127, 0.3578, 0.4403, 0.2159, 0.4104, 0.5977, 0.1134, 0.2907, 0.0421,
        0.2199, 0.2082, 0.3536, 0.1855, 0.2196, 0.2562, 0.2508, 0.2590, 0.1134,
        0.3568, 0.3619, 0.2733, 0.0750, 0.1360, 0.0264, 0.0258, 0.0473, 0.0466,
        0.0207, 0.1480, 0.2098, 0.2023, 0.1372, 0.2047, 0.0977, 0.1373, 0.3205,
        0.2326, 0.4069, 0.0393, 0.3620, 0.0597, 0.4921, 0.1714, 0.5164, 0.1255,
        0.3487, 0.1293, 0.1063, 0.2206, 0.1164, 0.1535, 0.0546, 0.1304, 0.0360,
        0.2004, 0.1092, 0.1482, 0.1537, 0.1302, 0.4048, 0.2068, 0.3328],
       dtype=torch.float64)


tensor(0.2280, dtype=torch.float64)

In [7]:
meter = DetectionAPMeter(80, algorithm='AUC', nproc=1)
for detection in save_data:
    if len(detection['labels']) != 0:
        mapped = torch.tensor(label_map_vec(detection['labels']))
    else:
        mapped = detection['labels']
    meter.append(detection['scores'], mapped, detection['binary_labels'])

meter.num_gt = num_gt.tolist()
ap, precs_alg, recalls_alg = meter.eval()

print(ap)
print(ap.mean().item())

tensor([0.3635, 0.2763, 0.1595, 0.2645, 0.4005, 0.4064, 0.4683, 0.3123, 0.2414,
        0.0088, 0.5267, 0.0862, 0.1576, 0.3932, 0.2055, 0.2816, 0.2840, 0.3419,
        0.3127, 0.3578, 0.4403, 0.2159, 0.4104, 0.5977, 0.1134, 0.2907, 0.0421,
        0.2199, 0.2082, 0.3536, 0.1855, 0.2196, 0.2562, 0.2508, 0.2590, 0.1134,
        0.3568, 0.3619, 0.2733, 0.0750, 0.1360, 0.0264, 0.0258, 0.0473, 0.0466,
        0.0207, 0.1480, 0.2098, 0.2023, 0.1372, 0.2047, 0.0977, 0.1373, 0.3205,
        0.2326, 0.4069, 0.0393, 0.3620, 0.0597, 0.4921, 0.1714, 0.5164, 0.1255,
        0.3487, 0.1293, 0.1063, 0.2206, 0.1164, 0.1535, 0.0546, 0.1304, 0.0360,
        0.2004, 0.1092, 0.1482, 0.1537, 0.1302, 0.4048, 0.2068, 0.3328],
       dtype=torch.float64)
0.2280052756898808


In [8]:
len(precs_alg)

80

In [37]:
recalls_alg[50] == recalls_alg[50].max()

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True])

In [40]:
recalls_alg[50]

tensor([0.0145, 0.0290, 0.0435, 0.0580, 0.0580, 0.0580, 0.0580, 0.0725, 0.0870,
        0.1014, 0.1159, 0.1304, 0.1449, 0.1594, 0.1594, 0.1594, 0.1594, 0.1594,
        0.1594, 0.1594, 0.1594, 0.1739, 0.1739, 0.1739, 0.1739, 0.1739, 0.1739,
        0.1884, 0.1884, 0.1884, 0.1884, 0.1884, 0.2029, 0.2029, 0.2029, 0.2029,
        0.2029, 0.2029, 0.2174, 0.2174, 0.2174, 0.2174, 0.2319, 0.2319, 0.2319,
        0.2319, 0.2319, 0.2319, 0.2319, 0.2319, 0.2319, 0.2319, 0.2464, 0.2609,
        0.2609, 0.2754, 0.2754, 0.2899, 0.3043, 0.3043, 0.3043, 0.3043, 0.3043,
        0.3043, 0.3043, 0.3188, 0.3188, 0.3188, 0.3188, 0.3188, 0.3188, 0.3188,
        0.3188, 0.3188, 0.3188, 0.3188, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333,
        0.3478, 0.3478, 0.3478, 0.3478, 0.3478, 0.3478, 0.3478, 0.3478, 0.3478,
        0.3478, 0.3478, 0.3478, 0.3478, 0.3478, 0.3478, 0.3478, 0.3478, 0.3478,
        0.3478, 0.3478, 0.3623, 0.3768, 0.3768, 0.3768, 0.3768, 0.3768, 0.3768,
        0.3768, 0.3768, 0.3768, 0.3768, 

In [39]:
precs_alg[50]

tensor([1.0000, 1.0000, 1.0000, 1.0000, 0.8000, 0.6667, 0.5714, 0.6250, 0.6667,
        0.7000, 0.7273, 0.7500, 0.7692, 0.7857, 0.7333, 0.6875, 0.6471, 0.6111,
        0.5789, 0.5500, 0.5238, 0.5455, 0.5217, 0.5000, 0.4800, 0.4615, 0.4444,
        0.4643, 0.4483, 0.4333, 0.4194, 0.4062, 0.4242, 0.4118, 0.4000, 0.3889,
        0.3784, 0.3684, 0.3846, 0.3750, 0.3659, 0.3571, 0.3721, 0.3636, 0.3556,
        0.3478, 0.3404, 0.3333, 0.3265, 0.3200, 0.3137, 0.3077, 0.3208, 0.3333,
        0.3273, 0.3393, 0.3333, 0.3448, 0.3559, 0.3500, 0.3443, 0.3387, 0.3333,
        0.3281, 0.3231, 0.3333, 0.3284, 0.3235, 0.3188, 0.3143, 0.3099, 0.3056,
        0.3014, 0.2973, 0.2933, 0.2895, 0.2987, 0.2949, 0.2911, 0.2875, 0.2840,
        0.2927, 0.2892, 0.2857, 0.2824, 0.2791, 0.2759, 0.2727, 0.2697, 0.2667,
        0.2637, 0.2609, 0.2581, 0.2553, 0.2526, 0.2500, 0.2474, 0.2449, 0.2424,
        0.2400, 0.2376, 0.2451, 0.2524, 0.2500, 0.2476, 0.2453, 0.2430, 0.2407,
        0.2385, 0.2364, 0.2342, 0.2321, 